In [2]:
%%time
import os
if not os.path.isdir('urban_oculus'):
  !git clone https://github.com/tgandor/urban_oculus
# first, this doesn't kill runtime
%run urban_oculus/snippets/get_coco_val2017_nounzip.ipynb
# this may kill runtime
# %run urban_oculus/snippets/get_detectron2.ipynb

Cloning into 'urban_oculus'...
remote: Enumerating objects: 446, done.
remote: Counting objects: 100% (446/446), done.
remote: Compressing objects: 100% (418/418), done.
remote: Total 446 (delta 127), reused 326 (delta 25), pack-reused 0
Receiving objects: 100% (446/446), 5.76 MiB | 4.11 MiB/s, done.
Resolving deltas: 100% (127/127), done.
Archive:  annotations_trainval2017.zip
  inflating: datasets/coco/annotations/instances_train2017.json  
  inflating: datasets/coco/annotations/instances_val2017.json  
  inflating: datasets/coco/annotations/captions_train2017.json  
  inflating: datasets/coco/annotations/captions_val2017.json  
  inflating: datasets/coco/annotations/person_keypoints_train2017.json  
  inflating: datasets/coco/annotations/person_keypoints_val2017.json  
--2020-04-15 17:42:46--  http://images.cocodataset.org/zips/val2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.216.251.148
Connecting to images.cocodataset.org (images.cocodataset.org)|52.216.

In [3]:
!unzip -o val2017.zip -d datasets/coco 2>&1 | tqdm --total 5000 >/dev/null
!du -h datasets/coco/val2017

5002it [00:06, 770.15it/s]              
788M	datasets/coco/val2017


In [4]:
!mogrify -format jp2 -quality 50 -verbose datasets/coco/val2017/*.jpg

mogrify-im6.q16: unable to open image `jp2:': No such file or directory @ error/blob.c/OpenBlob/2701.
mogrify-im6.q16: unrecognized image format `jp2' @ error/mogrify.c/MogrifyImageCommand/4927.


In [6]:
!mogrify -list format | grep 2000

           and supplement 61 which adds JPEG-2000 encoding.


In [0]:
quality = 25

In [9]:
!apt search openjpeg

Sorting... Done
Full Text Search... Done
libopenjp2-7-dev/bionic-updates,bionic-security 2.3.0-2build0.18.04.1 amd64
  development files for OpenJPEG, a JPEG 2000 image library



In [6]:
%%time
print('unpacking')
!unzip -o val2017.zip -d datasets/coco 2>&1 | tqdm --total 5000 >/dev/null
!du -h datasets/coco/val2017
print(f'degrading quality to {quality}')
!mogrify -verbose -quality {quality} -format jp2 datasets/coco/val2017/*.jpg # 2>&1 | tqdm --total 10000 >/dev/null
!du -h datasets/coco/val2017

unpacking
5001it [00:06, 732.83it/s]              
788M	datasets/coco/val2017
degrading quality to 25
mogrify-im6.q16: unable to open image `jp2:': No such file or directory @ error/blob.c/OpenBlob/2701.
mogrify-im6.q16: unrecognized image format `jp2' @ error/mogrify.c/MogrifyImageCommand/4927.
788M	datasets/coco/val2017
CPU times: user 78.8 ms, sys: 27.9 ms, total: 107 ms
Wall time: 11.8 s


In [0]:
%%time
import detectron2.model_zoo
from detectron2.config import get_cfg
from detectron2.data import build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.engine import DefaultPredictor
cfg = get_cfg()
cfg.merge_from_file(detectron2.model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
cfg.MODEL.WEIGHTS = "detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl"
predictor = DefaultPredictor(cfg)
data_loader = build_detection_test_loader(cfg, 'coco_2017_val')
evaluator = COCOEvaluator('coco_2017_val', cfg, False)
results = inference_on_dataset(predictor.model, data_loader, evaluator)
results

Loading and preparing results...
DONE (t=0.04s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=17.77s).
Accumulating evaluation results...
DONE (t=2.43s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.282
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.428
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.310
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.124
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.309
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.404
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.241
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.337
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.341
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1

In [0]:
result_file = f'coco_val2017_rcnn_3x_q_{quality}.json'
import json

with open(result_file, 'w') as f:
  f.write(json.dumps(results['bbox']))

try:
  from google.colab import files
  files.download(result_file)
except ImportError:
  # running locally
  from IPython.core.display import display, HTML
  display(HTML(f'Results: <a href="{result_file}" target="_blank">{result_file}</a>'))

In [1]:
!ls

sample_data
